# Cosas por revisar

- Cuando se aplica el zoom no se hace en la posicion que tiene el mouse sobre la imagen zoomeada, sino sobra la imagen original, eso quiere decir que so hago click sobre la derecha primero se va a zoomear correctamente, pero si despues hago click, en la imagen con un primer zoom, sobre la izquierda (que sería el centro de la imagen original) se va a ir a un zoom mas pero sobre la izquierda de la imagen original, no de la zoomeada (que, de nuevo, sería el centro de la imagen original). Hacer la prueba si no se entiende. Pienso que una solucion puede ser que cuando se aplique el zoom se guarden las coordenadas de los extremos donde se hizo zoom en una variable current_position y que la posicion x e y del mouse pueda referenciarse a esa current_position, pero hoy que hacer bien la matematica porque necesita un escalamiento, asi rapido creo que seria por ejemplo: new_left = current_position[0] + x/zoom_rate - width/zoom_rate*2

- Agregar el historial de posiciones para aplicar IA y aplicar gestos. Tener en cuenta que se puede submuestrear los datos para no recargar al sistema, tal vez 1 o 2 posiciones por segundo alcanza

- Utilizar en conjunto con el mouse, porque solo se probaron los cambios de imagenes. Ver si hace falta mejorar la velocidad (por ejemplo, se puede agregar un condicional que solo transforme a la imagen si hubo un click, porque ahora el cv2.warpAfine se esta ejecutando todas las veces, y es una multiplicacion de matrices..)

- Revisar resolucion de la ventana. Otra cosa: el flip se puede sacar y usar el war affine haciendo que left se haga right y right se haga left (digamos que simplemente hay que cambiar las coordenadas en init_position. Tambien habría que ver como dejar todo listo para poder abrir la ventana para claibrar el zoom y los parametros de sensibilidad, etc. y después poder cerrar la ventana y que el tracking siga funcionando.

- Threshold adaptativo: ver si conviene o no.

- Mejorar el temblor del puntero viendo si se puede hacer que el threshold de movimiento dependa de la escala de cada movimiento, es decir, si hago un movimiento MUY grande en x  y uno MUY pequeño en y, que solo genere el movimiento en x.

# Imports

In [1]:
import cv2
from IPython.display import clear_output
import time
from PIL import Image
from io import BytesIO
import IPython.display
import numpy as np
from matplotlib import pyplot as plt
import mouse

# Functions


## Zoom Functions

In [2]:
#Function to be called on mouse callback
def mousecall(event,x,y,flags,param):
    global zoom_rate
    global M
    
    if event == cv2.EVENT_LBUTTONDOWN:
        zoom_rate += 1
        M = apply_zoom(x,y, zoom_rate, False)
    elif event == cv2.EVENT_RBUTTONDOWN:
        zoom_rate = max(1, zoom_rate-1)
        M = apply_zoom(x,y, zoom_rate, True)

#I think this function is used to "detect" the mouse button release, it was used like this in the example I used as guide
def mousenone(event,x,y,flags,param):

    return 0

#This function gets the Transformation Matrix to apply to the image
def apply_zoom(x, y, zoom_rate, is_zoom_out):
    global width
    global height
    
    cv2.setMouseCallback('frame', mousenone)
    
    new_left = x-width/(zoom_rate*2)
    new_right = x+width/(zoom_rate*2)
    new_down = y+height/(zoom_rate*2) #take into consideration that the image is drawn from the upper left, so up is diff and down is sum
    new_up = y-height/(zoom_rate*2)

    new_position = np.float32([[new_left,new_up],[new_right,new_up],[new_left,new_down],[new_right,new_down]])
    
    matrix = cv2.getPerspectiveTransform(new_position,init_position)
    
    return matrix

## Alarm Functions

In [3]:
def check_alarms(frame, image_returned, mouse_coords):
    check_source(image_returned)
    check_reflex(frame)
    check_centroid_colour(frame, mouse_coords)
    
def raise_alarms():
    global source_flag
      
    if reflex_flag_counter > 40:
        print("Can't detect reflex point")
    if source_flag:
        print("Can't get source of image")
    if bright_flag_counter > 40:
        print("Too much bright in the background")   
    if centroid_surroundings_flag_counter> 40:
        print("There are too many reflective stickers")  
    
def check_reflex(frame): #sobre este check ahora probamos tanto el reflex como el brillo de fondo
    global reflex_flag_counter
    global bright_flag_counter
    
    colour_values, colour_counts = np.unique(frame, return_counts = True)
    colour_counts_total = np.sum(colour_counts)  
    
    for i in range(colour_values.shape[0]):
        if colour_values[i] == 0:
            if colour_counts[i]/colour_counts_total > 0.99:
                reflex_flag_counter += 1
            else:
                reflex_flag_counter = 0
        
        else:
            if colour_counts[i]/colour_counts_total > 0.01:  #División de los blancos sobre el total de pixels.
                bright_flag_counter += 1     
            else:                                            #Elegí 1% porque es el complemento del 99% definido en reflex
                bright_flag_counter = 0
         
        
def check_source(image_returned):
    global source_flag
    
    source_flag = not image_returned
    
    
def check_centroid_colour(frame, mouse_coords):       #falta agregarle la parte de deteccion de contornos   
    global centroid_surroundings_flag_counter
    
    
#    print("estoy en check_centroid_colour y el centroide es color: ", pointer_colour)
    
    cX=mouse_coord[0]
    cY=mouse_coord[1]
     
    recorte = frame[max(cY-20, 0): min(cY+20, height), max(cX-20, 0): min(cX+20, width)]

    colour_values, colour_counts = np.unique(recorte, return_counts = True)    
    colour_counts_total = np.sum(colour_counts)   
    

    
    
    print(colour_values , colour_counts)              #Prueba: borrar cuando terminemos de hacer la función
#    plt.imshow(recorte)                               #Prueba: borrar cuando terminemos de hacer la función
#    plt.show()                                        #Prueba: borrar cuando terminemos de hacer la función

                
                
                
    if colour_values[0] == 0:                                   # calculo el porcentaje de negros en la
        blacks_percentage = colour_counts[0]/colour_counts_total     # imagen recortada.
    else:
        blacks_percentage = 0 # será 0 en caso de que el subíndice 0 de color values sea un 1, porque no existen
                              # puntos negros
        

#    print("el porcentaje de negros es: ", blacks_percentage)  #veo el porcentaje de píxeles negros.
 

    
    
    
    
    if pointer_colour == 0 and ( 0.7 < blacks_percentage or blacks_percentage < 0.35):

        centroid_surroundings_flag_counter += 1
    else:
 
        centroid_surroundings_flag_counter = 0
        
        

    
    
  
       

## Image Processing Functions

In [4]:
#This function gets the frame from the camera and apply the transformations, also applies the movement to the mouse
def get_frame(cap, mouse_coord):
    global alarm_check_starttime
    
    ret, frame = cap.read()
    
    print(frame.shape)
    if ret:
        frame = cv2.warpPerspective(frame, M, (int(width), int(height)))

        frame = cv2.flip(frame, 1)

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) #CV works with BGR instead of RGB (its weird I know), we switch to it and then back for it to be easy to work with

        frame = color_extract(frame)
        
        mouse_coord = mouse_movement(detect_centroid(frame), mouse_coord)
    
        #frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR) #back to BGR
    
    check_alarms(frame, ret, mouse_coord)
    raise_alarms()
    
    return frame, mouse_coord

#This function creates a new 1D array that has 255 where all the pixel values on the three colour depths lie bewtween each
#limit in lower and upper, and 0 otherwise (so a pixel will be 255 if lower < pixel value < upper on ALL colour matrix)
def color_extract(image):
    
    #Masking 255,144,104
    lower = np.array([230,230,230])
    upper = np.array([255,255,255])
    
    res = cv2.inRange(image, lower, upper)
    
    return res

#This function detects the centroid position
#Detect centroid
def detect_centroid(res):
    global pointer_colour
    M = cv2.moments(res)
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
    else:
        cX, cY = 0, 0
    
    pointer_colour = res[cY, cX]                # almaceno el color del pixel sobre el que está el centroide
    
    print("el color sobre el cursor es: ", pointer_colour) 
    
# Chequeo si el color del centroide es negro y el porcentaje de blancos es bajo
# para activar una alarma

#    if pointer_color == 0 and blacks_percentage > 0.5:  # considerando que el objeto reflectante tiene aproximadamente
                                                         # 500-1000 píxeles de tamaño, tomo como valor arbitrario 800
                                                         # píxeles de tamaño, lo que representa el 50% del recorte.
                                                         # si la cantidad de negros es menor a ese porcentaje, alarma.
        
    
    
    cv2.rectangle(res, (cX - 1, cY - 1), (cX + 1, cY + 1), (255, 255, 255), -1)
    cv2.putText(res, "centroid", (cX - 25, cY - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    return [cX,cY]


## Mouse Functions

In [5]:
#This function moves the mouse to the new position
def mouse_movement(newPos, oldPos):
    
    #Avoind jerking when the point is lost
    if newPos[0]==0 and newPos[1]==0 :
        newPos = oldPos
    
    #hist_points_cent.append(newPos)  #History of centroid movements for gestures
    
    #calculate Movement Delta
    deltaX = (newPos[0]-oldPos[0])
    deltaY = (newPos[1]-oldPos[1])
    
    #Aplicamos un delta, si el movimiento es menor a ese delta no se mueve.
    if abs(deltaX)>delta_threshold or abs(deltaY)>delta_threshold:
        #print("Moved")
        mouse.move(mouse_sensibility(deltaX,'x'), mouse_sensibility(deltaY,'y'), absolute=False, duration=0)
    
    return newPos

def mouse_sensibility(delta, axis):
    assert axis == "x" or axis == "y"
    if abs(delta) > sensibility_threshold:
        return delta*sensibility_dict[axis]
    else:
        return delta

# Main

In [6]:
#Image Params
height = 480
width = 640
zoom_rate = 1
M = np.array([[1,0,0],[0,1,0],[0,0,1]], dtype = np.float32)

init_position = np.float32([[0,0],[width,0],[0,height],[width,height]]) #Initial position for the corners of the image

#Flags
reflex_flag_counter = 0
source_flag = False
bright_flag_counter = 0
centroid_surroundings_flag_counter = 0


#h_center = width/2
#v_center = height/2
#left = h_center - width/(2*zoom_rate)
#right = h_center + width/(2*zoom_rate)
#upper = v_center - height/(2*zoom_rate)
#lower = h_center + height/(2*zoom_rate)
#zoom_position = np.float32([[left,upper],[right,upper],[left,lower],[right,lower]])
#
#M = cv2.getPerspectiveTransform(zoom_position,init_position)

#Mouse Params
mouse_coord = [1,1]
delta_threshold = 1 #Threshold to interpret a centroid movement as a mouse movement

smoothness = 5

sensibility_dict = {"x" : 1,
                    "y" : 1} #mouse sensibility
sensibility_threshold = 10 #mouse sensibility threshold

#Historical points for gestures detection
#hist_points_cent = []
#hist_points_mouse = []

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 10000)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 10001)
    
cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
cv2.resizeWindow('frame', width, height)

try:
    while(True):

        cv2.setMouseCallback('frame', mousecall)

        frame, mouse_coord = get_frame(cap, mouse_coord)

        if not source_flag:
            cv2.imshow('frame',frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        elif cv2.waitKey(1) & 0xFF == ord('r'):
            zoom_rate = 1
            M = np.array([[1,0,0],[0,1,0],[0,0,1]], dtype = np.float32)

        clear_output(wait=True)
    cap.release()
    cv2.destroyAllWindows()
    
except KeyboardInterrupt:    
    cap.release()
    cv2.destroyAllWindows()

(480, 640, 3)
el color sobre el cursor es:  255
[255] [1600]
Too much bright in the background


# Links utiles:

- https://gist.github.com/nochekaiser/ab4b3b8b8ceaad7db7c0c303b0ff661d
- https://docs.opencv.org/3.4/db/d5b/tutorial_py_mouse_handling.html
- https://docs.opencv.org/3.4/da/d97/tutorial_threshold_inRange.html
- https://docs.opencv.org/3.4/d7/dfc/group__highgui.html
- https://docs.opencv.org/3.4/d2/de8/group__core__array.html
- https://www.programcreek.com/python/example/70462/cv2.setMouseCallback